In [1]:
from models.DAX import DAX_quantile_regression, DAX_baseline
from models.energy import energy_models
from functions import get_energy,get_DAX,merge_submissions,check_submission
from functions.prepare_data import split_time
from functions.evaluation import evaluate_horizon
from tqdm import tqdm
from functions import evaluate_dax
import numpy as np

In [2]:
from functions import naive_ensemble

In [3]:
daxdata = get_DAX.get()

In [4]:
base = DAX_baseline.DAX_baseline(daxdata)

In [5]:
base

,forecast_date,target,horizon,q0.025,q0.25,q0.5,q0.75,q0.975
0,2024-01-31,DAX,1 day,-3.084808,-0.515136,0.073395,0.718274,2.719772
1,2024-01-31,DAX,2 day,-4.002208,-0.767884,0.097835,0.987322,3.570492
2,2024-01-31,DAX,5 day,-4.720020,-0.869562,0.171704,1.255618,4.628697
3,2024-01-31,DAX,6 day,-5.401203,-0.937223,0.216215,1.451656,5.367396
4,2024-01-31,DAX,7 day,-5.908297,-1.008852,0.343478,1.607134,6.259934


In [6]:
quantile = DAX_quantile_regression.DAX_quantile_regression(daxdata)

In [7]:
base

,forecast_date,target,horizon,q0.025,q0.25,q0.5,q0.75,q0.975
0,2024-01-31,DAX,1 day,-3.084808,-0.515136,0.073395,0.718274,2.719772
1,2024-01-31,DAX,2 day,-4.002208,-0.767884,0.097835,0.987322,3.570492
2,2024-01-31,DAX,5 day,-4.720020,-0.869562,0.171704,1.255618,4.628697
3,2024-01-31,DAX,6 day,-5.401203,-0.937223,0.216215,1.451656,5.367396
4,2024-01-31,DAX,7 day,-5.908297,-1.008852,0.343478,1.607134,6.259934


In [8]:
quantile

,forecast_date,target,horizon,q0.025,q0.25,q0.5,q0.75,q0.975
0,2024-01-31,DAX,1 day,-2.311528,-0.502578,0.045126,0.547981,1.697530
1,2024-01-31,DAX,2 day,-2.177536,-0.341214,0.333601,0.843224,2.364898
2,2024-01-31,DAX,5 day,-2.560622,-0.702490,0.028832,0.551294,1.906253
3,2024-01-31,DAX,6 day,-2.066307,-0.185738,0.586287,1.140205,2.508612
4,2024-01-31,DAX,7 day,-1.686328,-0.055340,0.711045,1.223679,2.709402


In [9]:
df_dax = naive_ensemble.ensemble(base,quantile,0.8,0.2)

In [10]:
df_dax

,forecast_date,target,horizon,q0.025,q0.25,q0.5,q0.75,q0.975
0,2024-01-31,DAX,1 day,-2.930152,-0.512625,0.067741,0.684215,2.515324
1,2024-01-31,DAX,2 day,-3.637274,-0.682550,0.144988,0.958502,3.329373
2,2024-01-31,DAX,5 day,-4.288140,-0.836147,0.143129,1.114753,4.084208
3,2024-01-31,DAX,6 day,-4.734224,-0.786926,0.290229,1.389366,4.795639
4,2024-01-31,DAX,7 day,-5.063903,-0.818149,0.416991,1.530443,5.549828


In [11]:
energydata= get_energy.get()

  0%|          | 0/267 [00:00<?, ?it/s]/Users/stephantimpe/PycharmProjects/probabilistic-forecasting-challenge/functions/get_energy.py:28: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  energydata = pd.concat([energydata, pd.DataFrame(rawdata, columns=col_names)])
100%|██████████| 267/267 [00:27<00:00,  9.60it/s]


In [12]:
df_energy= energy_models.energy_XGBoost(energydata,date_str='2024-01-31')

In [13]:
df_energy

,forecast_date,target,horizon,q0.025,q0.25,q0.5,q0.75,q0.975
0,2024-01-31,energy,36 hour,63.441486,67.814606,68.914574,73.770218,76.208138
1,2024-01-31,energy,40 hour,61.259262,64.322090,64.568054,69.318184,71.906235
2,2024-01-31,energy,44 hour,60.863354,60.974594,61.808281,65.363388,67.771507
3,2024-01-31,energy,60 hour,57.142838,61.433403,61.500763,62.964813,64.108162
4,2024-01-31,energy,64 hour,53.561340,56.606220,57.705704,58.360950,61.336086
5,2024-01-31,energy,68 hour,53.569309,56.806168,57.884403,58.422134,59.887936


In [14]:
#Reorder quantiles to prevent quantile crossing

In [16]:
df = merge_submissions.merge(df_energy,df_dax)

  forecast_date      target  horizon     q0.025      q0.25       q0.5  \
0    2024-01-31         DAX    1 day  -2.930152  -0.512625   0.067741   
1    2024-01-31         DAX    2 day  -3.637274   -0.68255   0.144988   
2    2024-01-31         DAX    5 day   -4.28814  -0.836147   0.143129   
3    2024-01-31         DAX    6 day  -4.734224  -0.786926   0.290229   
4    2024-01-31         DAX    7 day  -5.063903  -0.818149   0.416991   
0    2024-01-31      energy  36 hour  63.441486  67.814606  68.914574   
1    2024-01-31      energy  40 hour  61.259262   64.32209  64.568054   
2    2024-01-31      energy  44 hour  60.863354  60.974594  61.808281   
3    2024-01-31      energy  60 hour  57.142838  61.433403  61.500763   
4    2024-01-31      energy  64 hour   53.56134   56.60622  57.705704   
5    2024-01-31      energy  68 hour  53.569309  56.806168  57.884403   
0    2024-01-31  infections   0 week         NA         NA         NA   
1    2024-01-31  infections   1 week         NA    

In [17]:
import pandas as pd

In [18]:
# ensure forecast_date is in correct format
df['forecast_date'] = pd.to_datetime(df['forecast_date'], format='%Y-%m-%d')

In [19]:
df

,forecast_date,target,horizon,q0.025,q0.25,q0.5,q0.75,q0.975
0,2024-01-31,DAX,1 day,-2.930152,-0.512625,0.067741,0.684215,2.515324
1,2024-01-31,DAX,2 day,-3.637274,-0.68255,0.144988,0.958502,3.329373
2,2024-01-31,DAX,5 day,-4.28814,-0.836147,0.143129,1.114753,4.084208
3,2024-01-31,DAX,6 day,-4.734224,-0.786926,0.290229,1.389366,4.795639
4,2024-01-31,DAX,7 day,-5.063903,-0.818149,0.416991,1.530443,5.549828
0,2024-01-31,energy,36 hour,63.441486,67.814606,68.914574,73.770218,76.208138
1,2024-01-31,energy,40 hour,61.259262,64.32209,64.568054,69.318184,71.906235
2,2024-01-31,energy,44 hour,60.863354,60.974594,61.808281,65.363388,67.771507
3,2024-01-31,energy,60 hour,57.142838,61.433403,61.500763,62.964813,64.108162
4,2024-01-31,energy,64 hour,53.56134,56.60622,57.705704,58.36095,61.336086


In [20]:
df.to_csv('../../forecasts/20240129_JonSnow.csv',index=False)

In [21]:
df

,forecast_date,target,horizon,q0.025,q0.25,q0.5,q0.75,q0.975
0,2024-01-31,DAX,1 day,-2.930152,-0.512625,0.067741,0.684215,2.515324
1,2024-01-31,DAX,2 day,-3.637274,-0.68255,0.144988,0.958502,3.329373
2,2024-01-31,DAX,5 day,-4.28814,-0.836147,0.143129,1.114753,4.084208
3,2024-01-31,DAX,6 day,-4.734224,-0.786926,0.290229,1.389366,4.795639
4,2024-01-31,DAX,7 day,-5.063903,-0.818149,0.416991,1.530443,5.549828
0,2024-01-31,energy,36 hour,63.441486,67.814606,68.914574,73.770218,76.208138
1,2024-01-31,energy,40 hour,61.259262,64.32209,64.568054,69.318184,71.906235
2,2024-01-31,energy,44 hour,60.863354,60.974594,61.808281,65.363388,67.771507
3,2024-01-31,energy,60 hour,57.142838,61.433403,61.500763,62.964813,64.108162
4,2024-01-31,energy,64 hour,53.56134,56.60622,57.705704,58.36095,61.336086


In [22]:
from datetime import timedelta, datetime

In [23]:
from functions import check_submission

In [24]:
check_submission.check_df(df)

Start checking...
---------------------------
Checking the Columns...
Checking type of columns...
----WARNING: Some elements in q0.025 column are not numeric. This may be fine if you only submit 2 out of 3 targets.

----WARNING: Some elements in q0.25 column are not numeric. This may be fine if you only submit 2 out of 3 targets.

----WARNING: Some elements in q0.5 column are not numeric. This may be fine if you only submit 2 out of 3 targets.

----WARNING: Some elements in q0.75 column are not numeric. This may be fine if you only submit 2 out of 3 targets.

----WARNING: Some elements in q0.975 column are not numeric. This may be fine if you only submit 2 out of 3 targets.

Checking if the Dates make sense...
Checking targets...
Skipping plausibility check for 'infections' as it contains NA values.
Skipping plausibility check for 'infections' as it contains NA values.
Skipping plausibility check for 'infections' as it contains NA values.
Checking quantiles...
Seems like you submitted 